In [1]:
import torch
import datasets
import transformers
import numpy as np
from seqeval.metrics import f1_score
import pandas as pd
import os
from torch.utils.data import DataLoader

In [2]:
from train_datasets import BPEDropoutTrainDataset

In [3]:
if torch.cuda.is_available():
    print("GPU is enabled.")
    print("device count: {}, current device: {}".format(torch.cuda.device_count(), torch.cuda.current_device()))
else:
    print("GPU is not enabled.")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

GPU is enabled.
device count: 6, current device: 0


In [4]:
cache_dir = "./cache"

# Load Data

In [5]:
dataset_path = "masakhaner"
language = "hau"
# dataset_path = "conll2003"
# language = None

# subword regularization params:
bpe_dropout_p = 0.1
# model_path = "xlm-roberta-base"
model_path = "Davlan/afro-xlmr-mini"

In [6]:
train_dataset = BPEDropoutTrainDataset(dataset_path, model_path, dataset_language=language, bpe_dropout_p=bpe_dropout_p, cache_dir=cache_dir, train=True)
test_dataset = BPEDropoutTrainDataset(dataset_path, model_path, dataset_language=language, cache_dir=cache_dir, train=False)


Found cached dataset masakhaner (/atlas2/u/xiluo/temp/cache/masakhaner/hau/1.0.0/e61b24903076a3af7682855beebb820ec64edad0d6787b148c473694592d10b3)


  0%|          | 0/3 [00:00<?, ?it/s]

Found cached dataset masakhaner (/atlas2/u/xiluo/temp/cache/masakhaner/hau/1.0.0/e61b24903076a3af7682855beebb820ec64edad0d6787b148c473694592d10b3)


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
tags = train_dataset.dset['train'].features["ner_tags"].feature
index2tag = {idx: tag for idx, tag in enumerate(tags.names)}
tag2index = {tag: idx for idx, tag in enumerate(tags.names)}
tags

ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-DATE', 'I-DATE'], id=None)

In [8]:
tokenizer = train_dataset.tokenizer

In [9]:
pd.DataFrame(
    [tokenizer.convert_ids_to_tokens(train_dataset[0]['input_ids']), train_dataset[0]['labels']],
    index=["tokens", "ner_tags"])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
tokens,<s>,▁A,▁sau,rari,▁cik,a,kken,▁raho,ton,▁wakil,in,▁Mur,yar,▁Amurka,▁Ibrahim,▁Abdul,',az,iz,</s>
ner_tags,-100,0,0,-100,0,-100,-100,0,-100,0,-100,3,-100,4,1,2,-100,-100,-100,-100


# Training

In [10]:
# Make debugging easier
%env TOKENIZERS_PARALLELISM=false

env: TOKENIZERS_PARALLELISM=false


In [11]:
training_args = transformers.TrainingArguments(
    output_dir = "./checkpoints/xlm-roberta-ner-yor-swreg",
    log_level = "error",
    num_train_epochs = 50,
    per_device_train_batch_size = 12,
    per_device_eval_batch_size = 12,
    evaluation_strategy = "epoch",
    fp16 = True,
    logging_steps = len(train_dataset),
    push_to_hub = False,
)

In [12]:
def metrics_func(eval_arg):
    preds = np.argmax(eval_arg.predictions, axis=2)
    batch_size, seq_len = preds.shape
    y_true, y_pred = [], []
    for b in range(batch_size):
        true_label, pred_label = [], []
        for s in range(seq_len):
            if eval_arg.label_ids[b, s] != -100:  # -100 must be ignored
                true_label.append(index2tag[eval_arg.label_ids[b][s]])
                pred_label.append(index2tag[preds[b][s]])
        y_true.append(true_label)
        y_pred.append(pred_label)
    return {"f1": f1_score(y_true, y_pred)}

In [13]:
data_collator = transformers.DataCollatorForTokenClassification(
    tokenizer,
    return_tensors="pt")

In [14]:
xlmr_config = transformers.AutoConfig.from_pretrained(
    model_path,
    num_labels=tags.num_classes,
    id2label=index2tag,
    label2id=tag2index
)

In [15]:
model = (transformers.RobertaForTokenClassification
         .from_pretrained(model_path, config=xlmr_config, cache_dir=cache_dir)
         .to(device))

Some weights of the model checkpoint at Davlan/afro-xlmr-mini were not used when initializing RobertaForTokenClassification: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at Davlan/afro-xlmr-mini and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use 

In [16]:
trainer = transformers.Trainer(
    model = model,
    args = training_args,
    data_collator = data_collator,
    compute_metrics = metrics_func,
    train_dataset = train_dataset,
    eval_dataset = test_dataset
)

In [17]:
trainer.train()

/deep/u/xiluo/anaconda3/envs/swreg/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/deep/u/xiluo/anaconda3/envs/swreg/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 15.74 GiB total capacity; 1001.81 MiB already allocated; 12.69 MiB free; 1.02 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF